In [63]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [65]:
path = '/content/drive/MyDrive/Data /ANN-3-kháng-sinh-CGB.xlsx'

In [66]:
excel = pd.ExcelFile(path)
sheets = excel.sheet_names
print(sheets)

['X', 'Xtest', 'Y', 'CGB 2 layers', 'CGB 5 layers', 'CGB 10 layers', 'CGB 40 layers']


In [67]:
X_train = pd.DataFrame(excel.parse(sheet_name= sheets[0], index_col= 0).values)
Y_train  = pd.DataFrame(excel.parse(sheet_name= sheets[2], index_col= 0).values)
X_test = pd.DataFrame(excel.parse(sheet_name= sheets[1], index_col= 0).values)
Y_test = pd.DataFrame(excel.parse(sheet_name= sheets[3], index_col= 0).values)

In [68]:
X_train.drop(0, inplace= True)
X_train.drop(X_train.tail(4).index, inplace = True)
print(X_train.shape)
print(X_train.corr())

(51, 61)
          0         1         2   ...        58        59        60
0   1.000000  0.999498  0.997246  ...  0.655760  0.645275  0.653579
1   0.999498  1.000000  0.998437  ...  0.662764  0.651794  0.660543
2   0.997246  0.998437  1.000000  ...  0.700576  0.690173  0.698346
3   0.988043  0.990536  0.996571  ...  0.753169  0.743385  0.751029
4   0.969721  0.973652  0.984605  ...  0.804962  0.795870  0.802965
..       ...       ...       ...  ...       ...       ...       ...
56  0.654978  0.661801  0.699719  ...  0.999732  0.997737  0.998518
57  0.648686  0.655325  0.693580  ...  0.998056  0.999449  0.997824
58  0.655760  0.662764  0.700576  ...  1.000000  0.997794  0.998579
59  0.645275  0.651794  0.690173  ...  0.997794  1.000000  0.997981
60  0.653579  0.660543  0.698346  ...  0.998579  0.997981  1.000000

[61 rows x 61 columns]


In [69]:
Y_train.drop(Y_train.head(3).index , inplace= True)
Y_train.drop(Y_train.tail(4).index, inplace= True)
Y_train = Y_train.iloc[:, 0:3]
Y_train.shape

(51, 3)

In [70]:
X_test.drop(0, inplace= True)
X_test.shape

(18, 61)

In [71]:
Y_test.drop(Y_test.head(6).index , inplace= True)
Y_test.drop(Y_test.tail(1).index, inplace= True)
y_test = Y_test.iloc[:, 0:3]
y_test[0]

6     27
7     19
8     21
9     23
10    25
11    27
12    25
13    17
14    19
15    21
16    23
17    25
18    25
19    17
20    19
21    21
22    23
23    27
Name: 0, dtype: object

Standardization data

In [72]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# scaler1 = StandardScaler()
# scaler2 = MinMaxScaler()
# scaler2.fit(X_train)
# X_train_scaler = scaler2.transform(X_train)
# X_test_scaler = scaler2.transform(X_test)
X_train_scaler = X_train
X_test_scaler = X_test


PCA 

In [73]:
from sklearn.decomposition import PCA
from math import sqrt

pca = PCA(n_components = 17)
X_train_pca = pca.fit_transform(X_train_scaler)
X_test_pca = pca.transform(X_test_scaler)

In [74]:
def MPE(act, predic):
  acts = np.asarray(act)
  predict = np.asarray(predic)
  res = np.empty(acts.shape)
  for j in range(acts.shape[0]):
    if acts[j] != 0:
      res[j] = (acts[j] - predict[j]) / acts[j]
    else:
      res[j] = predict[j] / np.mean(acts)
  return np.mean(res)

# 1. Linear Regression

In [88]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
regr = LinearRegression(fit_intercept = True)
regr.fit(X_train_pca, Y_train)
prediction1 = regr.predict(X_test_pca)

pre = pd.DataFrame(prediction1, columns = [0,1,2])
#print(pre[1])
#print(y_test)

print(sqrt(mean_squared_error(y_test[0], pre[0])))
print(sqrt(mean_squared_error(y_test[1], pre[1])))
print(sqrt(mean_squared_error(y_test[2], pre[2])))

print(100*r2_score(y_test[0], pre[0]))
print(100*r2_score(y_test[1], pre[1]))
print(100*r2_score(y_test[2], pre[2]))

print(100*MPE(y_test[0], pre[0]))
print(100*MPE(y_test[1], pre[1]))
print(100*MPE(y_test[2], pre[2]))

1.7553677578942153
1.357202801309906
0.9770219879069029
70.8767102448285
55.19460812175376
83.22769432686809
2.409942272294855
-7.369954402259355
6.632512641475535


#2. Lasso regression


In [76]:
from sklearn.linear_model import  Lasso

In [77]:
lasso_reg = Lasso(normalize = False)
lasso_reg.fit(X_train_pca, Y_train[2])
predict = lasso_reg.predict(X_test_pca)
print(sqrt(mean_squared_error(y_test[2], predict)))

print(r2_score(y_test[2], predict))

2.6031922225620088
-0.19068414220432528


#3. Ridge Regression

In [78]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

#4. Random forest regression

In [89]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(n_estimators=100, max_depth = 5, random_state = 0)
regr.fit(X_train_pca, Y_train)

predict_tree = regr.predict(X_test_pca)

pre_RT = pd.DataFrame(predict_tree, columns = [0,1,2])

print(sqrt(mean_squared_error(y_test[0], pre_RT[0])))
print(sqrt(mean_squared_error(y_test[1], pre_RT[1])))
print(sqrt(mean_squared_error(y_test[2], pre_RT[2])))

print(100*r2_score(y_test[0], pre_RT[0]))
print(100*r2_score(y_test[1], pre_RT[1]))
print(100*r2_score(y_test[2], pre_RT[2]))

print(100*MPE(y_test[0], pre_RT[0]))
print(100*MPE(y_test[1], pre_RT[1]))
print(100*MPE(y_test[2], pre_RT[2]))

3.0095268475021917
1.2974279662613086
1.0848798245916058
14.394702507263135
59.054394733155135
79.32015120644708
5.134625174789835
0.15247437747437656
-11.132913226177124


# 2. XGBoost

In [80]:
from xgboost import XGBRegressor

In [81]:
model = XGBRegressor()
model.fit(X_train_pca, Y_train[0])

predict_Tree = model.predict(X_test_pca)

print(sqrt(mean_squared_error(y_test[0], predict_Tree)))
print(r2_score(y_test[0], predict_Tree))

[03:38:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
1.5789082601240092
0.7643768321996337


In [90]:
from sklearn.multioutput import MultiOutputRegressor

multioutputregressor = MultiOutputRegressor(XGBRegressor(objective='reg:linear')).fit(X_train_pca, Y_train)
predictxgb = multioutputregressor.predict(X_test_pca)

pre1 = pd.DataFrame(predictxgb, columns = [0,1,2])
print(sqrt(mean_squared_error(y_test[0], pre1[0])))
print(sqrt(mean_squared_error(y_test[1], pre1[1])))
print(sqrt(mean_squared_error(y_test[2], pre1[2])))

print(r2_score(y_test[0], pre1[0]))
print(r2_score(y_test[1], pre1[1]))
print(r2_score(y_test[2], pre1[2]))

print(100*MPE(y_test[0], pre1[0]))
print(100*MPE(y_test[1], pre1[1]))
print(100*MPE(y_test[2], pre1[2]))

[03:47:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:47:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:47:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
1.5789082601240092
1.3393327364161622
1.191962226315609
0.7643768321996337
0.5636673078507015
0.7503629287071055
3.2017003299839564
-1.155465405232073
-5.203679093608151


# 3. Locally Estimated Scatterplot Smoothing (LOESS)


# 4. ANN


In [83]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from tensorflow.keras.optimizers import SGD
from keras import layers

In [93]:
def ANN(i, backsize, epo ):
  # create ANN model
  model = Sequential()

  # defining the Input layer and First hidden layer, both the same!
  model.add(Dense(units = 32, input_dim = X_train_pca.shape[1], kernel_initializer='normal', activation= 'relu'))

  # after the first layer we don't have to specify input_dim as keras configure it automatically
  model.add(Dense(units = 16, kernel_initializer = 'normal', activation = 'linear'))
  model.add(Dense(units = 8, kernel_initializer = 'normal', activation = 'linear'))
  # Since we will be predicting a single number
  model.add(Dense(1, kernel_initializer='normal'))

  # Compiling the model
  model.compile(loss='mean_squared_error', optimizer= 'adam')
  
  # Fitting the ANN to the Training set
  y_train = np.asarray(Y_train[i]).astype(np.float32)
  #print(y_train)
  model.fit(X_train_pca, y_train ,batch_size = backsize , epochs = epo, verbose=0)

  predic = model.predict(X_test_pca)
  #print(predic)
  print(sqrt(mean_squared_error(y_test[i], predic)))
  print(r2_score(y_test[i], predic))
  print("MPE")
  print(100*MPE(y_test[i], predic))

batck_size = 10, epochs  = 50

In [100]:
batch_size_trial = 10
epochs_trial = 1000

ANN(0, batch_size_trial, epochs_trial)
ANN(1, batch_size_trial, epochs_trial)
ANN(2, batch_size_trial, epochs_trial)
            

1.9427083646557128
0.643286605602841
MPE
2.065603497127692
1.3548621263022276
0.5534902045515204
MPE
-7.750104347036944
0.7538899626832314
0.9001380560898125
MPE
-0.8866331850488981
